In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import scipy.signal
import numpy as np
import seaborn as sns
plt.rcParams["figure.figsize"]=[15,5]

In [2]:
raw_sp = pd.read_csv('sp_jan.csv', usecols =['timestamp', 'data.A1'])
raw_sp.dropna(axis=0, inplace=True)
raw_sp.sort_values(by=['timestamp'])
raw_sp.reset_index(inplace=True)
raw_sp['timestamp'] =  pd.to_datetime(raw_sp['timestamp'])

In [3]:
test_sp=raw_sp
sptimethresh=12

In [4]:
#Get boards from Screenprinter
test_sp=test_sp[['data.A1','timestamp']]
# events=scipy.signal.find_peaks(test_sp['data.A1'], height=(2.22, 5), width=1)

In [5]:
test_sp['sum']=pd.Series.to_frame(test_sp['data.A1'].rolling(12, center=True).sum())

In [6]:
%matplotlib qt

In [7]:
plt.plot(test_sp['timestamp'], test_sp['sum'])
plt.plot(test_sp['timestamp'], test_sp['data.A1']*10)

In [8]:
plt.plot(test_sp['timestamp'], test_sp['sum'])

In [12]:
sptimethresh=12
printing_delays_raw=scipy.signal.find_peaks(test_sp['sum'], height=(24, 30), distance=sptimethresh, width=1)
printing_delays_raw_df=pd.DataFrame({"sample_number":printing_delays_raw[0], "working_time":printing_delays_raw[1]['widths']})
cleaning_delays_raw=scipy.signal.find_peaks(test_sp['sum'], height=60, distance=sptimethresh, width=1)
cleaning_delays_raw_df=pd.DataFrame({"sample_number":cleaning_delays_raw[0], "working_time":cleaning_delays_raw[1]['widths']})

In [13]:
# test_sp['detection']=0
# test_sp['detection'][(test_sp['sum']>=24) & (test_sp['sum']<=30) ]=1
# test_sp['detection'][test_sp['sum']>=60]=2

/home/raghu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/raghu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [14]:
SP_events=pd.DataFrame({"timestamp":test_sp.iloc[printing_delays_raw_df.sample_number].timestamp, "event":1, "working_time":printing_delays_raw_df.working_time.tolist()})
cleanings=pd.DataFrame({"timestamp":test_sp.iloc[cleaning_delays_raw_df.sample_number].timestamp, "event":2, "working_time":cleaning_delays_raw_df.working_time.tolist()})
# idles=pd.DataFrame({"timestamp":test_sp.iloc[idle_delays_raw_df.sample_number].timestamp, "event":0, "working_time":idle_delays_raw_df.working_time.tolist()})

SP_events.reset_index(inplace=True)
cleanings.reset_index(inplace=True)
# idles.reset_index(inplace=True)

In [15]:
for x, row in SP_events.iterrows():
    SP_events.ix[x,'energy']=test_sp.ix[int(printing_delays_raw[1]['left_ips'][x]):int(printing_delays_raw[1]['right_ips'][x]), 'data.A1'].sum()*230/3600000

# for index, x in enumerate(printing_delays_raw[0]):
#     SP_events.ix[index,'energy']=np.trapz(test_sp.ix[int(printing_delays_raw[1]['left_ips'][index]):int(printing_delays_raw[1]['right_ips'][index]), 'data.A1'].tolist()*230/3600000)

/home/raghu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  


In [16]:
for x, row in cleanings.iterrows():
    cleanings.ix[x,'energy']=test_sp.ix[int(cleaning_delays_raw[1]['left_ips'][x]):int(cleaning_delays_raw[1]['right_ips'][x]), 'data.A1'].sum()*230/3600000

# for index,x in enumerate(cleaning_delays_raw[0]):
#     cleanings.ix[index,'energy']=np.trapz(test_sp.ix[int(cleaning_delays_raw[1]['left_ips'][index]):int(cleaning_delays_raw[1]['right_ips'][index]), 'data.A1'].tolist()*230)
    

/home/raghu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  


In [17]:
# for x, row in idles.iterrows():
#     idles.ix[x,'energy']=test_sp.ix[int(idle_delays_raw[1]['left_ips'][x]):int(idle_delays_raw[1]['right_ips'][x]), 'data.A1'].sum()*230/3600000

In [18]:
SP_events.index=SP_events.timestamp
SP_events.drop('timestamp', axis=1, inplace=True)
SP_events.drop('index', axis=1, inplace=True)

cleanings.index=cleanings.timestamp
cleanings.drop('timestamp', axis=1, inplace=True)
cleanings.drop('index', axis=1, inplace=True)

# idles.index=idles.timestamp
# idles.drop('timestamp', axis=1, inplace=True)
# idles.drop('index', axis=1, inplace=True)


SP_events=SP_events.append(cleanings)
# SP_events=SP_events.append(idles)

In [19]:
sns.distplot(SP_events.working_time[SP_events['event']==1])
plt.title('SP Printing Times Histogram')

/home/raghu/anaconda3/lib/python3.6/site-packages/scipy/stats/stats.py:1713: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return np.add.reduce(sorted[indexer] * weights, axis=axis) / sumval


Text(0.5, 1.0, 'SP Printing Times Histogram')

In [20]:
sns.distplot(SP_events.working_time[SP_events['event']==2])
plt.title('SP Cleaning Times Histogram')

Text(0.5, 1.0, 'SP Cleaning Times Histogram')

In [21]:
print('Printing time Mode: ', SP_events.working_time[SP_events['event']==1].mode().mean())

Printing time Mode:  12.96700976109289


In [22]:
print('Cleaning time Mode: ', SP_events.working_time[SP_events['event']==2].mode().mean())

Cleaning time Mode:  12.350014895841353


In [23]:
# SP_events.to_csv('using_trap.csv')

In [30]:
cleaning_delays_raw[1]

{'peak_heights': array([76.18, 72.33, 70.57, 72.17, 69.8 , 72.24, 73.38, 77.51, 73.11,
        73.23, 76.55, 72.02, 76.03, 70.08, 73.14, 71.02, 76.39, 71.32,
        76.65, 70.19, 69.49, 70.44, 72.82, 67.  , 73.52, 71.84, 74.26,
        64.53, 70.38, 75.98, 72.2 , 76.65, 77.5 , 74.79, 71.91, 70.41,
        72.69, 73.23, 73.35, 77.18, 70.91, 68.03, 72.26]),
 'prominences': array([54.55, 50.67, 48.69, 50.3 , 47.96, 50.6 , 51.71, 61.4 , 51.38,
        52.46, 55.93, 51.06, 54.98, 46.96, 52.04, 49.82, 55.1 , 49.88,
        55.37, 48.82, 48.03, 49.03, 51.44, 45.35, 52.02, 50.22, 52.93,
        42.83, 48.91, 54.36, 50.38, 55.15, 57.02, 53.17, 50.12, 48.61,
        51.02, 51.76, 51.77, 55.79, 49.44, 46.51, 51.32]),
 'left_bases': array([22914, 29708, 30007, 30007, 32288, 32373, 32373, 22914, 33953,
        34681, 34681, 39040, 39040, 39636, 39952, 40283, 39040, 41501,
        39040, 41940, 42745, 41940, 41940, 43588, 41940, 44732, 41940,
        45443, 46240, 41940, 46847, 41940, 39040, 48125,

In [31]:
plt.plot(test_sp.timestamp, test_sp.detection)
plt.plot(test_sp.timestamp, test_sp['sum'])

In [29]:
SP_events[SP_events.event==1]

,event,working_time,energy
timestamp,,,
2019-01-02 05:20:41.111115,1,16.586123,0.002300
2019-01-02 05:21:29.991422,1,12.435687,0.001692
2019-01-02 05:23:52.747297,1,4.672222,0.000737
2019-01-02 05:24:37.703624,1,8.347243,0.001277
2019-01-02 05:24:58.905420,1,7.631132,0.001177
2019-01-02 05:26:30.983379,1,16.717014,0.002361
2019-01-02 05:27:55.581802,1,8.315972,0.001209
2019-01-02 05:28:42.039630,1,12.395092,0.001926
2019-01-02 05:32:02.769066,1,12.865236,0.001959


In [27]:
SP_events[SP_events.event==1].shape[0]

168

In [28]:
SP_events[SP_events.event==2].shape[0]

43